In [0]:
!pip install numpy==1.23.0
!pip install xarray
!pip install rioxarray
!pip install geopy

In [0]:
# restart the python kernel to import xarray correctly!
dbutils.library.restartPython()

In [0]:
import numpy as np
import pandas as pd
import datetime as dt
import pickle

import xarray as xr
import rioxarray
import rasterio
from rasterio.windows import Window, from_bounds
from rasterio.warp import Resampling
from rasterio.vrt import WarpedVRT

### Create "y" from survey data

In [0]:
# read in
df = pd.read_stata('/dbfs/FileStore/Myanmar_Survey_ML/data/survey/Assets_household_level.dta')

In [0]:
# paths
dirpath = '/dbfs/FileStore/Myanmar_Survey_ML/data/geo'
# this is the reference grid 
refpath = f'netcdf:{dirpath}/landcover/C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1-002.nc:lccs_class'

# myanmar bounding box
min_lon = 92.3032344909 
min_lat = 9.93295990645 
max_lon = 101.180005324 
max_lat = 28.335945136 

with rasterio.open(refpath) as src:
    # clip to myanmar
    win = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    win = Window(*[max(0, v) for v in win.flatten()])
    vrt_options = {
        # "crs": rasterio.crs.CRS.from_epsg(4326),  # the standard one -- any way to check the netcdf file?
        "transform": src.window_transform(win),
        "width": int(win.width),
        "height": int(win.height),
    }
    
vrt_options.update({'resampling':Resampling.bilinear})

# open transformed and clipped to myanmar
with rasterio.open(refpath) as src:
    with WarpedVRT(src, **vrt_options) as vrt:
        da = rioxarray.open_rasterio(vrt)

In [0]:
# convert survey's lat/lon to closest lat/lon in landcover
locations = np.array([[df.loc[i,'s0q22'], df.loc[i,'s0q23']] for i in df.index])
locs = da.sel(x=locations[:,1], y=locations[:,0], method='nearest')

In [0]:
# put it back in the survey data
df = pd.concat([df, pd.Series(locs.x.values, name='lon'), pd.Series(locs.y.values, name='lat')], axis=1)

###### changeable variables!

In [0]:
poverty_line = 1302.951
fgt_a1 = 0.25
fgt_a2 = 0.5  # change as needed
fgt_a3 = 0.75
################ CHECK CORRECT WEIGHT AND EXPENDITURE COLUMNS #######################
weight = 'hhweight'
expenditure = 'r_totex_pad_v3'

In [0]:
# those who are over the poverty line have negative numbers, all negatives are made 0 thru clip
# those who are under the poverty line have a positive number (float)
df = pd.concat([df, pd.Series((poverty_line - df[expenditure]).clip(0) / poverty_line, name='y')], axis=1)

# change month to datetime and cleaning
df.loc[:,'s0q20_mm'] = df['s0q20_mm'].apply(lambda x: dt.date(2017, int(x), 1))
d = df[['s0q20_mm', 'lat', 'lon', weight, 'y']]
d = d.rename(columns={'s0q20_mm':'time', weight:'weight'})

# convert 'poor' to 1 or fgta
d['y0'] = np.where(d["y"], 1.0, 0.0)
d['ya_25'] = np.where(d["y"], d["y"] ** fgt_a1, 0.0)
d['ya_50'] = np.where(d["y"], d["y"] ** fgt_a2, 0.0)
d['ya_75'] = np.where(d["y"], d["y"] ** fgt_a3, 0.0)

# get weighted average by lat/lon location and month
ds = d.groupby(['time', 'lat', 'lon']).apply(
    lambda x: pd.Series(
                [   
                    np.average(x['y0'], weights=x["weight"]),
                    np.average(x['y0']),
                    np.average(x['ya_25'], weights=x["weight"]),
                    np.average(x['ya_50'], weights=x["weight"]),
                    np.average(x['ya_75'], weights=x["weight"]),
                    np.average(x['ya_50']),
                ],
                index=('y0', 'y0_nw', 'ya_25', 'ya_50', 'ya_75', 'ya_50_nw')
            )
        ).reset_index()

Note on weighted/unweighted averages: there seems to be little difference!  
For y0, there are only 5 cases where the absolute difference between weighted and unweighted averages are > 0.01

In [0]:
ds

In [0]:
# save as pandas pickle
with open('/dbfs/FileStore/Myanmar_Survey_ML/data/survey/y_lcvr_ref_panda.pickle', 'wb') as handle:
    pickle.dump(ds, handle, protocol=pickle.HIGHEST_PROTOCOL)

### ACLED feature engineering

In [0]:
# read in acled data
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

database_host = dbutils.secrets.get(scope='warehouse_scope', key='database_host')
database_port = dbutils.secrets.get(scope='warehouse_scope', key='database_port')
user = dbutils.secrets.get(scope='warehouse_scope', key='user')
password = dbutils.secrets.get(scope='warehouse_scope', key='password')

database_name = "UNDP_DW_CRD"
table = "dbo.CRD_ACLED"
url = f"jdbc:sqlserver://{database_host}:{database_port};databaseName={database_name};"

df_all = (spark.read
      .format("com.microsoft.sqlserver.jdbc.spark")
      .option("url", url)
      .option("dbtable", table)
      .option("user", user)
      .option("password", password)
      .load()
    ) 

In [0]:
# filter to year and country
df = df_all.filter((df_all.ACLED_Year==2017) & (df_all.CountryFK==187))
display(df.limit(10))

In [0]:
# convert
acled = df.toPandas()
acled = acled[['TimeFK_Event_Date', 'ACLED_Event_Type', 'ACLED_Latitude', 'ACLED_Longitude' , 'ACLED_Geo_Precision', 'ACLED_Fatalities']]
# datetime to the first of the month
acled['time'] = acled['TimeFK_Event_Date'].apply(lambda x: str(x))
acled['time'] = acled['time'].apply(lambda x: dt.date(int(x[:4]), int(x[4:6]), 1))

In [0]:
# convert lcvr to pandas for easy looping
lcvr = da.to_dataframe().reset_index()
lcvr = lcvr[~np.isnan(lcvr.lccs_class)]
lcvr = lcvr.drop(['band','spatial_ref'], axis=1)
lcvr = lcvr.rename(columns={'y':'lat','x':'lon', 'lccs_class':'landcover'})
lcvr.head()

In [0]:
# there doesn't seem to be a way of clipping thru a radius in xarray
# see: https://stackoverflow.com/questions/74657517/how-to-select-x-grid-points-from-specific-grid-location-in-xarray
# this code is somewhat hacky, but minimizes the double for-loop as much as possible

from geopy.distance import geodesic
from geopy.distance import distance

col = []
for i, arow in acled.iterrows():
    # max distance (of influence) depends on the precision of the location
    if arow['ACLED_Geo_Precision'] == 1:
        mx = 5
    elif arow['ACLED_Geo_Precision'] == 2:
        mx = 20
    else:
        mx = 50

    coord1 = (arow['ACLED_Latitude'], arow['ACLED_Longitude'])
    dst = distance(kilometers=mx*1.05)
    n = dst.destination(point=coord1, bearing=0)[0] #get latitude for north-bound
    e = dst.destination(point=coord1, bearing=90)[1] #get longitude for east-bound
    s = dst.destination(point=coord1, bearing=180)[0] #get latitude for south-bound
    w = dst.destination(point=coord1, bearing=-90)[1] #get latitude for west-bound

    # filter lat-lon to reduce compute time
    lcvr_filter = lcvr.loc[(lcvr['lat'] < n) & (lcvr['lat'] > s) & (lcvr['lon'] < e) & (lcvr['lon'] > w), :]
    
    near = []
    # get exact distance and save if within max
    for j, wor in lcvr_filter.iterrows():
        coord2 = (wor['lat'], wor['lon'])
        dist = geodesic(coord1, coord2).km
        if dist <= mx:
            near.append((wor['lat'], wor['lon']))    
    col.append(near)

# save as column
acled['lcvr'] = col

In [0]:
# explode the list into separate rows
a = acled.explode('lcvr')

In [0]:
# number of events, normalized by geo precision
def event_count(sub):
    far = sub[sub['ACLED_Geo_Precision']==3].shape[0] / 3
    med = sub[sub['ACLED_Geo_Precision']==2].shape[0] / 2
    near = sub[sub['ACLED_Geo_Precision']==1].shape[0]
    return far + med + near

# number of fatalities, normalized by geo precision
def fatal_count(sub):
    far = sub.loc[sub['ACLED_Geo_Precision']==3, 'ACLED_Fatalities'].sum() / 3
    med = sub.loc[sub['ACLED_Geo_Precision']==2, 'ACLED_Fatalities'].sum() / 2
    near = sub.loc[sub['ACLED_Geo_Precision']==1, 'ACLED_Fatalities'].sum()
    return far + med + near

In [0]:
# groupby lat/lon coordinates and month - get 'normalized' event counts
ev = a.groupby(['lcvr','time']).apply(lambda sub: event_count(sub)).reset_index()
ev = ev.rename(columns={0:'event_count'})

In [0]:
# groupby lat/lon coordinates and month - get 'normalized' fatality counts
ft = a.groupby(['lcvr','time']).apply(lambda sub: fatal_count(sub)).reset_index()
ft = ft.rename(columns={0:'fatal_count'})

In [0]:
# merge together
mrg = pd.merge(ev, ft)
mrg['lat'] = mrg['lscn'].apply(lambda x: x[0])
mrg['lon'] = mrg['lscn'].apply(lambda x: x[1])
mrg = mrg[['lat','lon','time','event_count','fatal_count']]
mrg

In [0]:
# save as pandas pickle
with open('/dbfs/FileStore/Myanmar_Survey_ML/data/survey/acled_lcvr_ref_panda.pickle', 'wb') as handle:
    pickle.dump(mrg, handle, protocol=pickle.HIGHEST_PROTOCOL)